### Analyse Exploratoire des Données (EDA) pour les données TSLA

In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from datetime import datetime

In [17]:
# Configuration du style
plt.style.use('seaborn-v0_8')  # Style valide de matplotlib
sns.set_theme(style="whitegrid")

In [19]:
def load_data(file_path):
    """Charge et prépare les données"""
    # Lecture du CSV en sautant les 2 premières lignes d'en-tête
    df = pd.read_csv(file_path, skiprows=2)
    
    # Affichage des colonnes pour déboguer
    print("Colonnes disponibles:", df.columns.tolist())
    
    # Renommer les colonnes selon la structure du fichier
    df.columns = ['Date', 'Close', 'High', 'Low', 'Open', 'Volume']
    
    # Conversion de la colonne Date en datetime
    df['Date'] = pd.to_datetime(df['Date'])
    
    # Définition de Date comme index
    df.set_index('Date', inplace=True)
    
    # Vérification des colonnes
    required_columns = ['Close', 'Open', 'High', 'Low', 'Volume']
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        raise ValueError(f"Colonnes manquantes dans le DataFrame: {missing_columns}")
    
    return df

In [20]:
# 1. Analyse de la volatilité
def calculate_volatility(df, window=20):
    """Calcule la volatilité sur une fenêtre glissante"""
    returns = df['Close'].pct_change()
    volatility = returns.rolling(window=window).std() * np.sqrt(252)  # Annualisée
    return volatility

In [21]:
# 2. Calcul des rendements
def calculate_returns(df):
    """Calcule les rendements journaliers et cumulés"""
    daily_returns = df['Close'].pct_change()
    cumulative_returns = (1 + daily_returns).cumprod()
    return daily_returns, cumulative_returns

In [22]:
# 3. Analyse des tendances
def analyze_trends(df):
    """Analyse les tendances avec les moyennes mobiles"""
    df['SMA_20'] = df['Close'].rolling(window=20).mean()
    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    return df

In [23]:
# 4. Analyse du volume
def analyze_volume(df):
    """Analyse le volume et sa relation avec le prix"""
    volume_ma = df['Volume'].rolling(window=20).mean()
    return volume_ma

In [24]:
# Exécution des analyses
def main(df):
    # Calcul des métriques
    volatility = calculate_volatility(df)
    daily_returns, cumulative_returns = calculate_returns(df)
    df = analyze_trends(df)
    volume_ma = analyze_volume(df)

    # 1. Graphique des prix et volumes
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, 
                       vertical_spacing=0.03, 
                       row_heights=[0.7, 0.3])

    fig.add_trace(go.Candlestick(x=df.index,
                                open=df['Open'],
                                high=df['High'],
                                low=df['Low'],
                                close=df['Close'],
                                name='OHLC'),
                  row=1, col=1)

    fig.add_trace(go.Scatter(x=df.index, y=df['SMA_20'],
                            name='SMA 20',
                            line=dict(color='orange')),
                  row=1, col=1)

    fig.add_trace(go.Scatter(x=df.index, y=df['SMA_50'],
                            name='SMA 50',
                            line=dict(color='blue')),
                  row=1, col=1)

    fig.add_trace(go.Bar(x=df.index, y=df['Volume'],
                        name='Volume'),
                  row=2, col=1)

    fig.update_layout(
        title='Analyse Technique TSLA',
        yaxis_title='Prix ($)',
        yaxis2_title='Volume',
        xaxis_rangeslider_visible=False
    )

    # Sauvegarde du graphique
    fig.write_html('../../notebooks/eda_analysis/tsla_analysis.html')

    # 2. Distribution des rendements
    plt.figure(figsize=(12, 6))
    sns.histplot(daily_returns.dropna(), bins=50, kde=True)
    plt.title('Distribution des Rendements Journaliers')
    plt.xlabel('Rendement')
    plt.ylabel('Fréquence')
    plt.savefig('../../notebooks/eda_analysis/returns_distribution.png')
    plt.close()

    # 3. Volatilité
    plt.figure(figsize=(12, 6))
    volatility.plot()
    plt.title('Volatilité sur 20 jours')
    plt.xlabel('Date')
    plt.ylabel('Volatilité')
    plt.savefig('../../notebooks/eda_analysis/volatility.png')
    plt.close()

    # Statistiques descriptives
    stats = pd.DataFrame({
        'Statistique': [
            'Prix moyen',
            'Volatilité moyenne',
            'Rendement moyen journalier',
            'Rendement total',
            'Volume moyen',
            'Ratio de Sharpe (annualisé)'
        ],
        'Valeur': [
            df['Close'].mean(),
            volatility.mean(),
            daily_returns.mean(),
            (cumulative_returns.iloc[-1] - 1) * 100,
            df['Volume'].mean(),
            (daily_returns.mean() / daily_returns.std()) * np.sqrt(252)
        ]
    })
    
    stats.to_csv('../../notebooks/eda_analysis/statistics.csv', index=False)


In [25]:
if __name__ == "__main__":
    # Chargement des données
    df = load_data('../../data/raw/tsla.csv')
    
    # Exécution de l'analyse
    main(df) 

Colonnes disponibles: ['Date', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5']
